In [5]:
import numpy as np # linear algebra
import pandas as pd 

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from subprocess import check_output
print(check_output(["ls","CleanData/data_clean.csv"]).decode("utf8"))

CleanData/data_clean.csv



In [9]:
#Load Chicago Taxi trip data
%time
# We'll load some important columns only
df = pd.read_csv('CleanData/data_clean.csv',
                 usecols=['trip_start_timestamp', 'trip_end_timestamp', 'fare', 'pickup_latitude', 'pickup_longitude',
                          'dropoff_latitude', 'dropoff_longitude'])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [10]:
df.head()

,trip_start_timestamp,trip_end_timestamp,fare,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,12/31/2016 1:30,12/31/2016 1:45,6.25,41.901207,-87.676356,41.901207,-87.676356
1,12/31/2016 22:15,12/31/2016 22:30,6.75,41.922761,-87.699155,41.922761,-87.699155
2,12/31/2016 19:45,12/31/2016 20:00,4.75,41.965812,-87.655879,41.965812,-87.655879
3,12/31/2016 23:15,12/31/2016 23:30,8.00,41.885281,-87.657233,41.893216,-87.637844
4,12/31/2016 0:15,12/31/2016 0:30,8.25,41.899602,-87.633308,41.899602,-87.633308


In [11]:
df.shape

(28561, 7)

In [12]:
from bokeh.plotting import figure, output_notebook, show # bokeh plotting library
# We'll show the plots in the cells of this notebook
output_notebook()

Loading BokehJS ...

In [15]:
print(np.min(df['pickup_latitude']), np.min(df['pickup_longitude']))
print(np.max(df['pickup_latitude']), np.max(df['pickup_longitude']))

print(np.min(df['dropoff_latitude']), np.min(df['dropoff_longitude']))
print(np.max(df['dropoff_latitude']), np.max(df['dropoff_longitude']))

41.6738199 -87.91362459999998
42.00962288 -87.5514282
41.6738199 -87.91362459999998
42.00962288 -87.5349029


In [16]:
Chicago = x_range, y_range = ((41.68, 42.00), (-87.57, -87.91))

In [17]:
plot_width = int(750)
plot_height = int(plot_width//1.2)

def base_plot(tools='pan, wheel_zoom, reset', plot_width=plot_width, plot_height=plot_height, **plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
              x_range=x_range, y_range=y_range, outline_line_color=None,
              min_border=0, min_border_left=0, min_border_right=0,
              min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p

options = dict(line_color=None, fill_color='blue', size=5)

In [20]:
%%time
# let's plot 10k sample pickup
samples = df.sample(n=10000)
p = base_plot()

p.circle(x=samples['pickup_latitude'], y=samples['pickup_longitude'], **options)
show(p)

CPU times: user 65.5 ms, sys: 7.1 ms, total: 72.6 ms
Wall time: 73.6 ms


In [21]:
import datashader as ds
from datashader import transfer_functions as tr_fns
from datashader.colors import Greys9
Greys9_r = list(reversed(Greys9))[:2]

In [24]:
%%time
cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height, x_range=x_range, y_range=y_range)
agg = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude')
img = tr_fns.shade(agg, cmap=["white", 'darkblue'], how='linear')

img

CPU times: user 296 ms, sys: 14.7 ms, total: 311 ms
Wall time: 313 ms


In [30]:
from datashader.bokeh_ext import InteractiveImage
from functools import partial
from datashader.utils import export_image
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
from IPython.core.display import HTML, display

background = "black"
export = partial(export_image, export_path="export", background=background)
cm = partial(colormap_select, reverse=(background=="black"))

def create_image(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude')
    img = tr_fns.shade(agg, cmap=Hot, how='eq_hist')
    return tr_fns.dynspread(img, threshold=0.5, max_px=4)

p = base_plot(background_fill_color=background)
export(create_image(*NYC), "N_HOT")
InteractiveImage(p, create_image)

NameError: name 'IL' is not defined

In [28]:
from functools import partial

def create_image90(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude', ds.count('passenger_count'))
    img = tr_fns.shade(agg.where(agg > np.percentile(agg, 90)), cmap=inferno, how='eq_hist')
    return tr_fns.dynspread(img, threshold=0.3, max_px=4)
    
p = base_plot()
export(create_image(*CHI), "CHI_90th")
InteractiveImage(p, create_image90)

NameError: name 'CHI' is not defined

In [ ]:
def merged_images(x_range, y_range, w=plot_width, h=plot_height, how='log'):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    picks = cvs.points(df, 'pickup_longitude', 'pickup_latitude', ds.count('passenger_count'))
    drops = cvs.points(df, 'dropoff_longitude', 'dropoff_latitude', ds.count('passenger_count'))
    more_drops = tr_fns.shade(drops.where(drops > picks), cmap=["darkblue", 'cornflowerblue'], how=how)
    more_picks = tr_fns.shade(drops.where(picks > drops), cmap=["darkred", 'orangered'], how=how)
    img = tr_fns.stack(more_picks, more_drops)
    return tr_fns.dynspread(img, threshold=0.3, max_px=4)

p = base_plot(background_fill_color=background)
export(merged_images(*NYC), "NYCT_pickups_vs_drops")
InteractiveImage(p, merged_images)